<a href="https://colab.research.google.com/github/tavishcode/fyp/blob/master/make_train_and_test_sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, Activation, CuDNNGRU, GRU, Dense
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import scale, StandardScaler, RobustScaler
from collections import OrderedDict, defaultdict
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive')
plt.rcParams['figure.figsize'] = [10, 8]

Using TensorFlow backend.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_start = 0
train_end = 100
test_start = train_end
test_end = test_start + 100
ds = np.load('drive/My Drive/rescaled_dataset.npy')
train = ds[:, train_start:train_end]
test = ds[:, test_start:test_end]

In [15]:
train.shape

(27222, 100)

In [16]:
test.shape

(27222, 100)

In [23]:
# Making date features
df = pd.read_csv('drive/My Drive/train_2.csv', nrows=1)
df.columns

Index(['Page', '2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04',
       '2015-07-05', '2015-07-06', '2015-07-07', '2015-07-08', '2015-07-09',
       ...
       '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04', '2017-09-05',
       '2017-09-06', '2017-09-07', '2017-09-08', '2017-09-09', '2017-09-10'],
      dtype='object', length=804)

In [0]:
dates = df.columns.values[1:]
dts = []
for date in dates:
  dt = datetime.strptime(date, '%Y-%m-%d')
  # [year, month, day, weekday, is_weekend]
  dts.append([dt.year, dt.month, dt.day, dt.weekday(), 
              int(dt.weekday()//5 == 1)])

In [26]:
len(dts)

803

In [27]:
num_features = 1 + 5 + 3
train_set = np.zeros((train.shape[0], train.shape[1], num_features))
test_set = np.zeros((test.shape[0], test.shape[1], num_features))
for i in range(train.shape[0]):
  x_train = np.column_stack(( train[i], 
                              dts[train_start:train_end], 
                              np.full((train.shape[1]),np.sum(train[i])), 
                              np.full((train.shape[1]),np.median(train[i])),
                              np.full((train.shape[1]),np.std(train[i]))))
  x_test = np.column_stack((  test[i], 
                              dts[test_start:test_end], 
                              np.full((test.shape[1]),np.sum(train[i])), 
                              np.full((test.shape[1]),np.median(train[i])),
                              np.full((test.shape[1]),np.std(train[i]))))
  train_set[i, :, :] = x_train
  test_set[i, :, :] = x_test 
print(train_set.shape)
print(test_set.shape)

(27222, 100, 9)
(27222, 100, 9)


In [0]:
np.save('train_set.npy', train_set)
np.save('test_set.npy', test_set)